# Automação para baixar máscaras do Quick Annotator

Uma vez que o modelo de detecção do **Quick Annotator (QA)** é treinado e refinado com correções, é possível baixar as máscaras das imagens sem a necessidades que conferir a corretude das anotações. 

Por ser um trabalho mecânico que espera o programa fazer a predição das regiões de interece (*ROI*) e então baixa a máscara, foi criado uma automação que espera a predição ser feita para baixar a imagem e ir para a próxima imagem.

Existe um bug no programa que as vezes o estado da predição não progride e para "destravar" esse bug é preciso navegar de uma imagem para outra. Assim a automação espera um determinado tempo e, caso a predição não esteja pronta, faz esta navegação. Esse processo também e feito para que duas sejam carregadas ao mesmo tempo. Quando é identificado que a predição foi concluída a automação baixa a imagem, avança para a imagem seguinte duas vezes e por fim volta uma vez.

### Estrutura do Código
> 1. Conta quantas imagens tem na pasta destino
> 2. Baixa a máscara da imagem caso a flag que indica a predição esteja verde e avança para as próximas imagens.
> 3. Caso contrário espera a flag ficar verde.
> 4. Se a flag não ficar verde depois de 1 min avança para a próxima imagem e retorna para a imagem anterior.
> 5. Repete esse processo tantas vezes quantas imagens foram contadas na pasta.

**Obs:** para que a automação funcione é necessário alterar algumas configurações do navegador: mudar a forma de Download para 'Perguntar onde salvar cada arquivo antes de fazer download' e é necessário fazer o downloas manual da primeira predição.

### Importando Biblíotecas necessárias

In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
import pyautogui
import time
import os

### Drive do navegador

In [2]:
# Utiliza sempre o drive mais novo
service = Service(ChromeDriverManager().install())
browser = webdriver.Chrome(service=service)

[WDM] - Downloading: 100%|█████████████████████████████████████████████████████| 6.79M/6.79M [00:02<00:00, 3.33MB/s]


### Automação

In [3]:
# Coloque aqui o edereço do link da primeira imagem
browser.get('http://localhost:5555/GasHis%20Annotation/2017-06-10_15.51.33.ndpi.16.31268_27437.2048x2048.png/annotation')

O **xPath** é um *label* utilizado para encontrar o elemento de uma página. Para encontra-lo pode-se seguir o tutorial presente neste [link](https://escoladedados.org/tutoriais/xpath-para-raspagem-de-dados-em-html/#:~:text=O%20modo%20mais%20r%C3%A1pido%20de,a%20op%C3%A7%C3%A3o%20'Copiar%20XPath'.).

In [4]:
# xPaths
prediction_flag = '//*[@id="prediction-dot"]'
image_name = '//*[@id="image_name"]/text()'
download_bt = '//*[@id="title"]/div/div[1]/ul[1]/li[2]/a'
dl_results_bt = '//*[@id="title"]/div/div[1]/ul[1]/li[2]/ul/li[2]'
next_img_bt = '//*[@id="title"]/div/div[1]/ul[1]/li[4]/a'
previous_img_bt = '//*[@id="title"]/div/div[1]/ul[1]/li[3]/a'

In [5]:
# Navega para a próxima página e volta para página anterior
def next_and_previous(browser):
    url = browser.current_url # pega o url da página atual
    browser.get(url) # carrega página atual
    time.sleep(10)
    
    # avança mais uma imagem para carregar duas imagens
    browser.find_element('xpath', next_img_bt).click() # Next Image
    time.sleep(1)
     
    # verifica pop de mensagem de erro
    try:
    #identify element
        erro_msg = browser.find_element('class name','bootstrap-dialog-title').text
        while erro_msg == 'Error Message':
            pyautogui.click(2238, 612)
            erro_msg = browser.find_element('class name','bootstrap-dialog-title').text
    #NoSuchElementException thrown if not present
    except NoSuchElementException:
        print("Element does not exist")

    url = browser.current_url # pega o url da página atual
    browser.get(url) # carrega página atual
    time.sleep(10)
    
    # volta para imagem anterior
    browser.find_element('xpath', previous_img_bt).click() # Previous Image
    time.sleep(1)

In [8]:
# Aperta o ALT+TAB (necessário somente ao abrir o navegador)
pyautogui.hotkey('alt','tab')

# Conta quantas imagens existem na pasta
path = r"C:\Users\IA\Pictures\Datasets\GasHis\cancer_Sub_Ori_train2"
n_img = len(os.listdir(path))

# Verifica predição, Download mascara e navegação
for i in range(n_img-31):
    
    element = browser.find_element('xpath', prediction_flag)
    prediction = element.value_of_css_property('background-color')   
    time.sleep(2)

    count = 0
    
    # Verifica se a flag prediction ficou verde   
    while (prediction != 'rgba(0, 255, 0, 1)'): 
        count += 1
        
        url = browser.current_url # pega o url da página atual
        browser.get(url) # carrega página atual
        time.sleep(20)
        
        element = browser.find_element('xpath', prediction_flag)
        prediction = element.value_of_css_property('background-color')
        
        # a cada 1 min vai para próxima imagem e retorna (tratamento do bug)
        if ((count > 3) & (prediction != 'rgba(0, 255, 0, 1)')):
            time.sleep(2)
            next_and_previous(browser)
            count = 0
        
        
    # FAZER DOWNLOAD DA IMAGEM

    # Clicar nos botões
    browser.find_element('xpath', download_bt).click() # Download
    browser.implicitly_wait(1) # espera 

    browser.find_element('xpath', dl_results_bt).click() # DL Result
    browser.implicitly_wait(2) # espera 

    # Nome do Download
    download_name = browser.find_element('id', 'image_name').text
    print(f'{i} - {download_name}')
    download_name = download_name[:-4] + '_mask.png'
    time.sleep(5) # espera 

    # Alterar o nome do Downaload e salvar imagem
    pyautogui.press('backspace')
    pyautogui.write(download_name, interval=0.02)
    pyautogui.press('enter')
    time.sleep(5) # espera 

    # CARREGAR PRÓXIMAS IMAGENS

    # vai para a próxima imagem
    browser.find_element('xpath', next_img_bt).click() # Next Image
    time.sleep(1)

    next_and_previous(browser)
    
    url = browser.current_url # pega o url da página atual
    browser.get(url) # carrega página atual
    time.sleep(10)


0 - 2017-06-10_18.47.26.ndpi.17.48562_18196.2048x2048.png
Element does not exist
1 - 2017-06-10_18.59.38.ndpi.16.41035_17206.2048x2048.png
Element does not exist
2 - 2017-06-10_18.59.38.ndpi.16.40056_19511.2048x2048.png
Element does not exist
3 - 2017-06-10_18.59.38.ndpi.16.46298_13385.2048x2048.png
Element does not exist
4 - 2017-06-10_18.59.38.ndpi.16.43201_17445.2048x2048.png
Element does not exist
5 - 2017-06-10_18.59.38.ndpi.16.48632_14875.2048x2048.png
Element does not exist
6 - 2017-06-10_18.59.38.ndpi.16.46947_18595.2048x2048.png
Element does not exist
7 - 2017-06-10_19.10.40.ndpi.17.34532_24240.2048x2048.png
Element does not exist
8 - 2017-06-10_19.10.40.ndpi.17.33157_21760.2048x2048.png
Element does not exist
9 - 2017-06-10_19.10.40.ndpi.17.39735_27290.2048x2048.png
Element does not exist
10 - 2017-06-10_19.10.40.ndpi.17.37021_33891.2048x2048.png
Element does not exist
11 - 2017-06-10_19.33.20.ndpi.17.30321_17309.2048x2048.png
Element does not exist
12 - 2017-06-10_19.33.20.n

FailSafeException: PyAutoGUI fail-safe triggered from mouse moving to a corner of the screen. To disable this fail-safe, set pyautogui.FAILSAFE to False. DISABLING FAIL-SAFE IS NOT RECOMMENDED.

O script abaixo faz o download apenas de uma imagem com o nome especificado na variável `img_name`.

In [6]:
## FAZER DOWNLOAD DA IMAGEM
img_name = '2017-06-10_15.51.33.ndpi.16.31268_27437.2048x2048' + '.png'
url = 'http://localhost:5555/GasHis%20Annotation/' + img_name + '/annotation'
browser.get(url)


# Clicar nos botões
browser.find_element('xpath', download_bt).click() # Download
browser.implicitly_wait(1) # espera 

browser.find_element('xpath', dl_results_bt).click() # DL Resu
browser.implicitly_wait(2) # espera 

# Nome do Download
download_name = browser.find_element('id', 'image_name').text
print(download_name)
download_name = download_name[:-4] + '_mask.png'

# Alterar o nome do Downaload e clicar em salvar
pyautogui.press('backspace')
time.sleep(3) # espera 
pyautogui.write(download_name, interval=0.02)
pyautogui.press('enter')
browser.implicitly_wait(5) # espera 

2017-06-10_15.51.33.ndpi.16.31268_27437.2048x2048.png


In [12]:
# pegar a posição do botão da mensagem de erro
time.sleep(3)
pyautogui.position()

Point(x=2238, y=612)